# Clean GDSC dataset

In [61]:
import json
import pickle

import pandas as pd
import numpy as np

import rdkit
import rdkit.Chem as Chem
from rdkit.Chem import AllChem

In [17]:
datadir = "/Volumes/OXYTOCIN/datasets/gdsc"
response_fname = "GDSC2_fitted_dose_response_24Jul22.xlsx" # drug response data
expr_fname = "Cell_line_RMA_proc_basalExp.txt"             # gene expression data
cell_line_meta_fname = "Cell_Lines_Details.xlsx"           # cell line metadata
cmpds_fname = "Drug_list Jan 20 2023.csv"                  # compound metadata
pubchem_fname = "gdsc2_drug_pubchem_info.csv"              # pubchem info about compounds

In [71]:
def create_molecular_graph_from_smiles(smiles):
    mol = Chem.MolFromSmiles(smiles)
    starts = []
    ends = []
    for bond in mol.GetBonds():
        start = bond.GetBeginAtomIdx()
        end = bond.GetEndAtomIdx()
        
        # bidirectional
        starts.append(start)
        ends.append(end)
        starts.append(end)
        ends.append(start)
        
    bonds = np.array([starts, ends], dtype=np.int64)
    
    atoms = []
    for atom in mol.GetAtoms():
        features = [atom.GetAtomicNum()]
        atoms.append(features)
    
    atoms = np.array(atoms, dtype=np.int64)
    
    return atoms, bonds, mol

In [79]:
# read in compounds and filter for the ones we have external data (e.g. structure) for
cmpds = pd.read_csv(f"{datadir}/{cmpds_fname}", converters={"Drug Id": str})
# columns:
# ['Drug Id', 'Name', 'Synonyms', 'Targets', 'Target pathway', 'PubCHEM',
#  'Datasets', 'number of cell lines', 'Screening site']
cmpds = cmpds[cmpds["Datasets"] == "GDSC2"]
cmpds = cmpds[cmpds["PubCHEM"].notna()]
cmpds = cmpds[cmpds["PubCHEM"] != "None"]
cmpds = cmpds[cmpds["PubCHEM"] != "none"]
cmpds = cmpds[cmpds["PubCHEM"] != "several"]
cmpds["PubCHEM"] = cmpds["PubCHEM"].map(lambda f: f.split(",")[0])
cmpds = cmpds.sort_values(by="PubCHEM")

print(len(cmpds))

# read in pubchem additional info
pubchem = pd.read_csv(pubchem_fname, header=None, names=["pubchem_id", "pubchem_json"], converters={"pubchem_id": str})
pubchem_dict = {}
for _, row in pubchem.iterrows():
    pubchem_id = row["pubchem_id"]
    pubchem_json = row["pubchem_json"]
    pubchem_dict[pubchem_id] = json.loads(pubchem_json)

# add smiles string
cmpds["SMILES"] = cmpds.apply(lambda r: pubchem_dict[r.to_dict()["PubCHEM"]]["canonical_smiles"], axis=1)

cid_to_info = {}
for _, row in cmpds.iterrows():
    cid = row["Drug Id"]
    name = row["Name"]
    pubchem = row["PubCHEM"]
    smiles = row["SMILES"]
    
    atoms, bonds, mol = create_molecular_graph_from_smiles(smiles)
    
    cid_to_info[cid] = {
        "name": name,
        "pubchemid": pubchem,
        "smiles": smiles,
        "atoms": atoms,
        "bonds": bonds,
        "mol": mol
    }

186


In [80]:
# save chemical graphs
with open("drugid_to_molecular_graphs.pkl", "wb") as fout:
    pickle.dump(cid_to_info, fout)

In [38]:
# read in cell line gene expression data
gexpr = pd.read_csv(f"{datadir}/{expr_fname}", sep="\t")

# only keep genes with name
gexpr = gexpr.dropna(subset=["GENE_SYMBOLS"])

cosmicid_to_gexpr = {} # cosmicids to gene expression data
for cid in gexpr.columns:
    if cid in {"GENE_SYMBOLS", "GENE_title"}:
        continue
    cid_cleaned = cid.split(".", 1)[1]
    cosmicid_to_gexpr[cid_cleaned] = list(gexpr[cid])

In [89]:
with open("cell_line_gexpr.pkl", "wb") as fout:
    pickle.dump(cosmicid_to_gexpr, fout)

In [ ]:
# output mapping of gene index to gene symbol
symbols_list = list(gexpr["GENE_SYMBOLS"])
with open("gexpr_gene_symbols.pkl", "wb") as fout:
    pickle.dump(symbols_list, fout)
    
print(len(symbols_list))
# 17419 genes

In [82]:
# read in drug response data
dr = pd.read_excel(f"{datadir}/{response_fname}", converters={"COSMIC_ID": str, "DRUG_ID": str}) # drug response (dr)

In [83]:
# keep lines where
# - DATASET = GDSC2
# - COSMIC_ID in cosmicid_to_gexpr
# - DRUG_IG in cid_to_info.keys()
dr = dr[dr["DATASET"] == "GDSC2"]
dr = dr[dr["COSMIC_ID"].apply(lambda cosmic_id: cosmic_id in cosmicid_to_gexpr)]
dr = dr[dr["DRUG_ID"].apply(lambda drug_id: drug_id in cid_to_info)]
dr

,DATASET,NLME_RESULT_ID,NLME_CURVE_ID,COSMIC_ID,CELL_LINE_NAME,SANGER_MODEL_ID,TCGA_DESC,DRUG_ID,DRUG_NAME,PUTATIVE_TARGET,PATHWAY_NAME,COMPANY_ID,WEBRELEASE,MIN_CONC,MAX_CONC,LN_IC50,AUC,RMSE,Z_SCORE
0,GDSC2,401,18945558,683667,PFSK-1,SIDM01132,MB,1003,Camptothecin,TOP1,DNA replication,1046,Y,0.000100,0.1,-1.462148,0.930105,0.088999,0.432482
1,GDSC2,401,18945796,684052,A673,SIDM00848,UNCLASSIFIED,1003,Camptothecin,TOP1,DNA replication,1046,Y,0.000100,0.1,-4.869447,0.614932,0.111423,-1.420322
2,GDSC2,401,18946078,684057,ES5,SIDM00263,UNCLASSIFIED,1003,Camptothecin,TOP1,DNA replication,1046,Y,0.000100,0.1,-3.360684,0.790953,0.142754,-0.599894
3,GDSC2,401,18946335,684059,ES7,SIDM00269,UNCLASSIFIED,1003,Camptothecin,TOP1,DNA replication,1046,Y,0.000100,0.1,-5.045014,0.592624,0.135642,-1.515791
4,GDSC2,401,18946617,684062,EW-11,SIDM00203,UNCLASSIFIED,1003,Camptothecin,TOP1,DNA replication,1046,Y,0.000100,0.1,-3.741620,0.733992,0.128066,-0.807038
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
236903,GDSC2,401,19186874,1659818,MM1S,SIDM01265,MM,2359,GSK2830371,WIP1,Other,1094,Y,0.010005,10.0,4.290123,0.991533,0.020244,-0.962882
236904,GDSC2,401,19187483,1659928,SNU-175,SIDM00216,COREAD,2359,GSK2830371,WIP1,Other,1094,Y,0.010005,10.0,5.423870,0.985066,0.032710,0.001082
236905,GDSC2,401,19187936,1660034,SNU-407,SIDM00214,COREAD,2359,GSK2830371,WIP1,Other,1094,Y,0.010005,10.0,5.042005,0.970009,0.026729,-0.323597
236906,GDSC2,401,19188194,1660035,SNU-61,SIDM00194,COREAD,2359,GSK2830371,WIP1,Other,1094,Y,0.010005,10.0,6.130028,0.986298,0.027488,0.601490


In [87]:
# compare drug response cosmic ids with gene expression ids
drout = dr[["COSMIC_ID", "DRUG_ID", "LN_IC50"]].rename(columns={"COSMIC_ID": "cosmic_id", "DRUG_ID": "drug_id", "LN_IC50": "ln_IC50"})
print(len(set(drout["cosmic_id"]))) # 941 cell lines
drout

941


,cosmic_id,drug_id,ln_IC50
0,683667,1003,-1.462148
1,684052,1003,-4.869447
2,684057,1003,-3.360684
3,684059,1003,-5.045014
4,684062,1003,-3.741620
...,...,...,...
236903,1659818,2359,4.290123
236904,1659928,2359,5.423870
236905,1660034,2359,5.042005
236906,1660035,2359,6.130028


In [88]:
with open("drug_response_with_cell_line.pkl", "wb") as fout:
    pickle.dump(drout, fout)